In [47]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
from codigos import extract_features, cross_validation, grid_search, preprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import cv2
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
path = "C:/Users/PC/Documents/Eye_diseases_dataset/"

# Imagenes originales

## Vggnet16

In [5]:
x_train_vggnet, y_train_vggnet, x_test_vggnet, y_test_vggnet = extract_features.features_vgg16(path)

******************************************************************** 
Extracción de caracteristicas por medio de VGGNET16
******************************************************************** 
Labels: ['cataratas' 'glaucoma' 'normal' 'retinopatia']
Labels encoded: [0 1 2 3]
4/4 [==============================] - 10s 2s/step
******************************************************************** 
Extracción de caracteristicas por medio de VGGNET16 finalizada
******************************************************************** 


In [8]:
models_cv = cross_validation.compare_models(x_train_vggnet, y_train_vggnet, 5)

******************************************************************** 
Iniciando torneo de modelos con validación cruzada
******************************************************************** 
Modelo: Random Forest
Modelo: Naive Bayes
Modelo: SVM
Modelo: LGBM
Modelo: XGBoost
******************************************************************** 
Finalizando torneo de modelos con validación cruzada
******************************************************************** 


In [9]:
models_cv

,Model,Train Accuracy,Train precision_macro,Train recall_macro,Train f1_macro,Train AUC-ROC,Test Accuracy,Test precision_macro,Test recall_macro,Test f1_macro,Test AUC-ROC
0,Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000,0.600000,0.534480,0.437500,0.411632,0.728427
1,Naive Bayes,0.929688,0.911184,0.955469,0.929024,0.969911,0.481250,0.463116,0.398958,0.409685,0.596458
2,SVM,0.500000,0.125000,0.250000,0.166667,0.894525,0.500000,0.125000,0.250000,0.166667,0.761827
3,LGBM,1.000000,1.000000,1.000000,1.000000,1.000000,0.585417,0.501710,0.451042,0.439506,0.756615
4,XGBoost,1.000000,1.000000,1.000000,1.000000,1.000000,0.591667,0.511623,0.470833,0.461867,0.745747


In [10]:
muestras_en_intervalo = 50
n_iter = 10

learning_ra_0 = list(set([float(x) for x in np.linspace(start = 0.001, stop = 0.8, num = muestras_en_intervalo)]))
estimadores_0 = list(set([int(x) for x in np.linspace(start = 10, stop = 200, num = muestras_en_intervalo)]))
profundidad_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 20, num = muestras_en_intervalo)]))
min_data_le_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 0.5*x_train_vggnet.shape[0], num = muestras_en_intervalo)]))

parametros = {'learning_ra': learning_ra_0,
              'estimadores': estimadores_0,
              'profundidad': profundidad_0,
              'min_data_le': min_data_le_0
             }

In [11]:
ronda_0 = grid_search.train_models(x_train_vggnet, y_train_vggnet, x_test_vggnet, y_test_vggnet, n_iter, parametros)

******************************************************************** 
Iniciando busqueda de hiperparametros
******************************************************************** 
******************************************************************** 
Iteración: 0
******************************************************************** 
[LightGBM] [Warning] min_data_in_leaf is set=132, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=132
[LightGBM] [Warning] min_data_in_leaf is set=220, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=220
[LightGBM] [Warning] min_data_in_leaf is set=240, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=240
[LightGBM] [Warning] min_data_in_leaf is set=108, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=108
[LightGBM] [Warning] min_data_in_leaf is set=54, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=54
[LightGBM] [Warning] min_data_in_leaf is set=98, mi

In [12]:
ronda_0.head(5)

,parameters,modelo,auc_tr,accuracy_tr,precision_tr,recall_tr,f1_tr,Matriz_tr,auc_te,accuracy_te,precision_te,recall_te,f1_te,Matriz_te
0,"{'n_estimators': 91, 'max_depth': 4, 'min_data...",Lighgbm,1.000000,1.000000,1.000000,1.000000,1.000000,"[[80, 0, 0, 0], [0, 80, 0, 0], [0, 0, 240, 0],...",0.796819,0.658333,0.635819,0.554167,0.559000,"[[17, 1, 2, 0], [2, 7, 11, 0], [2, 2, 52, 4], ..."
1,"{'n_estimators': 165, 'max_depth': 8, 'min_sam...",RandomForest,0.922748,0.616667,0.343834,0.435417,0.377005,"[[61, 0, 19, 0], [7, 0, 73, 0], [5, 0, 235, 0]...",0.753597,0.616667,0.345000,0.441667,0.381250,"[[16, 0, 4, 0], [2, 0, 18, 0], [2, 0, 58, 0], ..."
2,"{'n_estimators': 118, 'max_depth': 4, 'min_chi...",XGBoost,1.000000,1.000000,1.000000,1.000000,1.000000,"[[80, 0, 0, 0], [0, 80, 0, 0], [0, 0, 240, 0],...",0.768361,0.675000,0.694318,0.562500,0.571341,"[[16, 1, 3, 0], [3, 7, 10, 0], [1, 4, 54, 1], ..."
3,"{'n_estimators': 72, 'max_depth': 3, 'min_data...",Lighgbm,0.987236,0.879167,0.906008,0.827083,0.853975,"[[77, 0, 3, 0], [3, 59, 17, 1], [1, 2, 236, 1]...",0.760333,0.633333,0.632450,0.516667,0.515812,"[[16, 0, 4, 0], [2, 4, 12, 2], [3, 1, 52, 4], ..."
4,"{'n_estimators': 137, 'max_depth': 1, 'min_sam...",RandomForest,0.785768,0.500000,0.125000,0.250000,0.166667,"[[0, 0, 80, 0], [0, 0, 80, 0], [0, 0, 240, 0],...",0.701819,0.500000,0.125000,0.250000,0.166667,"[[0, 0, 20, 0], [0, 0, 20, 0], [0, 0, 60, 0], ..."


In [13]:
ronda_0.to_csv("vggnet16/ronda_0.csv",sep=";")

## Resnet 

In [ ]:
x_train_resnet2, y_train_resnet2, x_test_resnet2, y_test_resnet2 = extract_features.features_resnet(path)

In [14]:
x_train_resnet, y_train_resnet, x_test_resnet, y_test_resnet = extract_features.features_resnet(path)

******************************************************************** 
Extracción de caracteristicas por medio de RESNET
******************************************************************** 
Labels: ['cataratas' 'glaucoma' 'normal' 'retinopatia']
Labels encoded: [0 1 2 3]
******************************************************************** 
Extracción de caracteristicas por medio de RESNET finalizada
******************************************************************** 


In [15]:
models_cv = cross_validation.compare_models(x_train_resnet, y_train_resnet, 5)

******************************************************************** 
Iniciando torneo de modelos con validación cruzada
******************************************************************** 
Modelo: Random Forest
Modelo: Naive Bayes
Modelo: SVM
Modelo: LGBM
Modelo: XGBoost
******************************************************************** 
Finalizando torneo de modelos con validación cruzada
******************************************************************** 


In [16]:
models_cv

,Model,Train Accuracy,Train precision_macro,Train recall_macro,Train f1_macro,Train AUC-ROC,Test Accuracy,Test precision_macro,Test recall_macro,Test f1_macro,Test AUC-ROC
0,Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000,0.595833,0.513346,0.439583,0.420717,0.760191
1,Naive Bayes,0.680729,0.655151,0.649219,0.648385,0.856578,0.570833,0.530232,0.502083,0.506953,0.747033
2,SVM,0.641146,0.772753,0.461719,0.439747,0.924073,0.602083,0.350906,0.411458,0.364389,0.777609
3,LGBM,1.000000,1.000000,1.000000,1.000000,1.000000,0.577083,0.536716,0.459375,0.450628,0.734440
4,XGBoost,1.000000,1.000000,1.000000,1.000000,1.000000,0.572917,0.498308,0.461458,0.451557,0.744180


In [18]:
muestras_en_intervalo = 50
n_iter = 10

learning_ra_0 = list(set([float(x) for x in np.linspace(start = 0.001, stop = 0.8, num = muestras_en_intervalo)]))
estimadores_0 = list(set([int(x) for x in np.linspace(start = 10, stop = 200, num = muestras_en_intervalo)]))
profundidad_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 20, num = muestras_en_intervalo)]))
min_data_le_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 0.5*x_train_resnet.shape[0], num = muestras_en_intervalo)]))

parametros = {'learning_ra': learning_ra_0,
              'estimadores': estimadores_0,
              'profundidad': profundidad_0,
              'min_data_le': min_data_le_0
             }

In [19]:
ronda_0 = grid_search.train_models(x_train_resnet, y_train_resnet, x_test_resnet, y_test_resnet, n_iter, parametros)

******************************************************************** 
Iniciando busqueda de hiperparametros
******************************************************************** 
******************************************************************** 
Iteración: 0
******************************************************************** 
[LightGBM] [Warning] min_data_in_leaf is set=132, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=132
[LightGBM] [Warning] min_data_in_leaf is set=220, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=220
[LightGBM] [Warning] min_data_in_leaf is set=240, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=240
[LightGBM] [Warning] min_data_in_leaf is set=108, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=108
[LightGBM] [Warning] min_data_in_leaf is set=54, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=54
[LightGBM] [Warning] min_data_in_leaf is set=98, mi

In [20]:
ronda_0.head(5)

,parameters,modelo,auc_tr,accuracy_tr,precision_tr,recall_tr,f1_tr,Matriz_tr,auc_te,accuracy_te,precision_te,recall_te,f1_te,Matriz_te
0,"{'n_estimators': 91, 'max_depth': 4, 'min_data...",Lighgbm,1.000000,1.000000,1.000000,1.000000,1.000000,"[[80, 0, 0, 0], [0, 80, 0, 0], [0, 0, 240, 0],...",0.803639,0.625000,0.563187,0.537500,0.541824,"[[16, 0, 4, 0], [1, 7, 7, 5], [3, 4, 48, 5], [..."
1,"{'n_estimators': 165, 'max_depth': 8, 'min_sam...",RandomForest,0.929948,0.620833,0.358866,0.431250,0.379995,"[[58, 0, 22, 0], [7, 0, 73, 0], [0, 0, 240, 0]...",0.839514,0.600000,0.344952,0.408333,0.360434,"[[13, 0, 7, 0], [1, 0, 19, 0], [1, 0, 59, 0], ..."
2,"{'n_estimators': 118, 'max_depth': 4, 'min_chi...",XGBoost,1.000000,1.000000,1.000000,1.000000,1.000000,"[[80, 0, 0, 0], [0, 80, 0, 0], [0, 0, 240, 0],...",0.800986,0.658333,0.647781,0.529167,0.538875,"[[14, 1, 4, 1], [2, 7, 10, 1], [3, 2, 55, 0], ..."
3,"{'n_estimators': 72, 'max_depth': 3, 'min_data...",Lighgbm,0.980554,0.877083,0.913859,0.821875,0.854054,"[[76, 0, 4, 0], [3, 59, 18, 0], [0, 1, 237, 2]...",0.811833,0.608333,0.544805,0.504167,0.503029,"[[15, 1, 4, 0], [2, 6, 11, 1], [3, 5, 49, 3], ..."
4,"{'n_estimators': 137, 'max_depth': 1, 'min_sam...",RandomForest,0.808220,0.500000,0.125000,0.250000,0.166667,"[[0, 0, 80, 0], [0, 0, 80, 0], [0, 0, 240, 0],...",0.780167,0.500000,0.125000,0.250000,0.166667,"[[0, 0, 20, 0], [0, 0, 20, 0], [0, 0, 60, 0], ..."


In [21]:
ronda_0.to_csv("resnet/ronda_0.csv",sep=";")

## NasNet 

In [22]:
x_train_nasnet, y_train_nasnet, x_test_nasnet, y_test_nasnet = extract_features.features_nasnet(path)

******************************************************************** 
Feature extraction using NASNET
******************************************************************** 
Found 480 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
4/4 [==============================] - 29s 7s/step
Labels: ['cataratas' 'glaucoma' 'normal' 'retinopatia']
Labels encoded: [0 1 2 3]
******************************************************************** 
Feature extraction finished
******************************************************************** 


In [23]:
models_cv = cross_validation.compare_models(x_train_nasnet, y_train_nasnet, 5)

******************************************************************** 
Iniciando torneo de modelos con validación cruzada
******************************************************************** 
Modelo: Random Forest
Modelo: Naive Bayes
Modelo: SVM
Modelo: LGBM
Modelo: XGBoost
******************************************************************** 
Finalizando torneo de modelos con validación cruzada
******************************************************************** 


In [24]:
models_cv

,Model,Train Accuracy,Train precision_macro,Train recall_macro,Train f1_macro,Train AUC-ROC,Test Accuracy,Test precision_macro,Test recall_macro,Test f1_macro,Test AUC-ROC
0,Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000,0.468750,0.179675,0.242708,0.182509,0.506970
1,Naive Bayes,0.382812,0.457993,0.411198,0.366498,0.660163,0.241667,0.224132,0.227083,0.175415,0.469078
2,SVM,0.501563,0.150401,0.253385,0.173659,0.389208,0.495833,0.124731,0.247917,0.165957,0.505330
3,LGBM,1.000000,1.000000,1.000000,1.000000,1.000000,0.429167,0.211818,0.237500,0.203587,0.504913
4,XGBoost,1.000000,1.000000,1.000000,1.000000,1.000000,0.418750,0.218160,0.238542,0.212333,0.501849


In [25]:
muestras_en_intervalo = 50
n_iter = 10

learning_ra_0 = list(set([float(x) for x in np.linspace(start = 0.001, stop = 0.8, num = muestras_en_intervalo)]))
estimadores_0 = list(set([int(x) for x in np.linspace(start = 10, stop = 200, num = muestras_en_intervalo)]))
profundidad_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 20, num = muestras_en_intervalo)]))
min_data_le_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 0.5*x_train_nasnet.shape[0], num = muestras_en_intervalo)]))

parametros = {'learning_ra': learning_ra_0,
              'estimadores': estimadores_0,
              'profundidad': profundidad_0,
              'min_data_le': min_data_le_0
             }

In [26]:
ronda_0 = grid_search.train_models(x_train_nasnet, y_train_nasnet, x_test_nasnet, y_test_nasnet, n_iter, parametros)

******************************************************************** 
Iniciando busqueda de hiperparametros
******************************************************************** 
******************************************************************** 
Iteración: 0
******************************************************************** 
[LightGBM] [Warning] min_data_in_leaf is set=132, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=132
[LightGBM] [Warning] min_data_in_leaf is set=220, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=220
[LightGBM] [Warning] min_data_in_leaf is set=240, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=240
[LightGBM] [Warning] min_data_in_leaf is set=108, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=108
[LightGBM] [Warning] min_data_in_leaf is set=54, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=54
[LightGBM] [Warning] min_data_in_leaf is set=98, mi

In [27]:
ronda_0.head(5)

,parameters,modelo,auc_tr,accuracy_tr,precision_tr,recall_tr,f1_tr,Matriz_tr,auc_te,accuracy_te,precision_te,recall_te,f1_te,Matriz_te
0,"{'n_estimators': 91, 'max_depth': 4, 'min_data...",Lighgbm,1.000000,1.000000,1.000000,1.0000,1.000000,"[[80, 0, 0, 0], [0, 80, 0, 0], [0, 0, 240, 0],...",0.482792,0.383333,0.237255,0.233333,0.221675,"[[2, 2, 16, 0], [4, 0, 14, 2], [7, 8, 41, 4], ..."
1,"{'n_estimators': 165, 'max_depth': 8, 'min_sam...",RandomForest,0.836201,0.500000,0.125000,0.2500,0.166667,"[[0, 0, 80, 0], [0, 0, 80, 0], [0, 0, 240, 0],...",0.531653,0.500000,0.125000,0.250000,0.166667,"[[0, 0, 20, 0], [0, 0, 20, 0], [0, 0, 60, 0], ..."
2,"{'n_estimators': 118, 'max_depth': 4, 'min_chi...",XGBoost,1.000000,1.000000,1.000000,1.0000,1.000000,"[[80, 0, 0, 0], [0, 80, 0, 0], [0, 0, 240, 0],...",0.550556,0.391667,0.174132,0.212500,0.183802,"[[0, 3, 15, 2], [3, 1, 14, 2], [6, 3, 45, 6], ..."
3,"{'n_estimators': 72, 'max_depth': 3, 'min_data...",Lighgbm,0.856424,0.541667,0.845005,0.3125,0.286495,"[[6, 1, 73, 0], [0, 6, 74, 0], [0, 0, 240, 0],...",0.497403,0.508333,0.501068,0.270833,0.213203,"[[0, 0, 20, 0], [0, 1, 19, 0], [0, 0, 59, 1], ..."
4,"{'n_estimators': 137, 'max_depth': 1, 'min_sam...",RandomForest,0.617976,0.500000,0.125000,0.2500,0.166667,"[[0, 0, 80, 0], [0, 0, 80, 0], [0, 0, 240, 0],...",0.488639,0.500000,0.125000,0.250000,0.166667,"[[0, 0, 20, 0], [0, 0, 20, 0], [0, 0, 60, 0], ..."


In [28]:
ronda_0.to_csv("nasnet/ronda_0.csv",sep=";")

# Imagenes preprocesadas

In [49]:
preprocessing.preprocesamiento_imagenes(path)

******************************************************************** 
Cargando imagenes
******************************************************************** 
Se cargaron 480 imágenes de train.
Se cargaron 120 imágenes de test.
******************************************************************** 
Carga de imagenes finalizada
******************************************************************** 


In [50]:
path_gray = "C:/Users/PC/Documents/Eye_diseases_dataset/gray"
path_equa = "C:/Users/PC/Documents/Eye_diseases_dataset/equalizer"

## Gray

## Vggnet16

In [51]:
x_train_vggnet, y_train_vggnet, x_test_vggnet, y_test_vggnet = extract_features.features_vgg16(path_gray)

******************************************************************** 
Extracción de caracteristicas por medio de VGGNET16
******************************************************************** 
Labels: ['cataratas' 'glaucoma' 'normal' 'retinopatia']
Labels encoded: [0 1 2 3]
4/4 [==============================] - 11s 3s/step
******************************************************************** 
Extracción de caracteristicas por medio de VGGNET16 finalizada
******************************************************************** 


In [52]:
models_cv = cross_validation.compare_models(x_train_vggnet, y_train_vggnet, 5)

******************************************************************** 
Iniciando torneo de modelos con validación cruzada
******************************************************************** 
Modelo: Random Forest
Modelo: Naive Bayes
Modelo: SVM
Modelo: LGBM
Modelo: XGBoost
******************************************************************** 
Finalizando torneo de modelos con validación cruzada
******************************************************************** 


In [53]:
models_cv

,Model,Train Accuracy,Train precision_macro,Train recall_macro,Train f1_macro,Train AUC-ROC,Test Accuracy,Test precision_macro,Test recall_macro,Test f1_macro,Test AUC-ROC
0,Random Forest,1.0000,1.000000,1.000000,1.000000,1.000000,0.591667,0.544863,0.429167,0.409963,0.717161
1,Naive Bayes,0.9375,0.926748,0.947396,0.934035,0.967738,0.518750,0.486891,0.417708,0.427418,0.610347
2,SVM,0.5000,0.125000,0.250000,0.166667,0.889945,0.500000,0.125000,0.250000,0.166667,0.742487
3,LGBM,1.0000,1.000000,1.000000,1.000000,1.000000,0.562500,0.457261,0.425000,0.407814,0.722031
4,XGBoost,1.0000,1.000000,1.000000,1.000000,1.000000,0.566667,0.471377,0.443750,0.429259,0.725590


In [54]:
muestras_en_intervalo = 50
n_iter = 10

learning_ra_0 = list(set([float(x) for x in np.linspace(start = 0.001, stop = 0.8, num = muestras_en_intervalo)]))
estimadores_0 = list(set([int(x) for x in np.linspace(start = 10, stop = 200, num = muestras_en_intervalo)]))
profundidad_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 20, num = muestras_en_intervalo)]))
min_data_le_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 0.5*x_train_vggnet.shape[0], num = muestras_en_intervalo)]))

parametros = {'learning_ra': learning_ra_0,
              'estimadores': estimadores_0,
              'profundidad': profundidad_0,
              'min_data_le': min_data_le_0
             }

In [55]:
ronda_0 = grid_search.train_models(x_train_vggnet, y_train_vggnet, x_test_vggnet, y_test_vggnet, n_iter, parametros)

******************************************************************** 
Iniciando busqueda de hiperparametros
******************************************************************** 
******************************************************************** 
Iteración: 0
******************************************************************** 
[LightGBM] [Warning] min_data_in_leaf is set=132, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=132
[LightGBM] [Warning] min_data_in_leaf is set=220, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=220
[LightGBM] [Warning] min_data_in_leaf is set=240, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=240
[LightGBM] [Warning] min_data_in_leaf is set=108, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=108
[LightGBM] [Warning] min_data_in_leaf is set=54, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=54
[LightGBM] [Warning] min_data_in_leaf is set=98, mi

In [56]:
ronda_0.head(5)

,parameters,modelo,auc_tr,accuracy_tr,precision_tr,recall_tr,f1_tr,Matriz_tr,auc_te,accuracy_te,precision_te,recall_te,f1_te,Matriz_te
0,"{'n_estimators': 91, 'max_depth': 4, 'min_data...",Lighgbm,1.000000,1.00000,1.000000,1.000000,1.000000,"[[80, 0, 0, 0], [0, 80, 0, 0], [0, 0, 240, 0],...",0.819125,0.641667,0.609719,0.554167,0.563222,"[[17, 0, 3, 0], [2, 6, 11, 1], [2, 2, 49, 7], ..."
1,"{'n_estimators': 165, 'max_depth': 8, 'min_sam...",RandomForest,0.907145,0.60625,0.327462,0.426042,0.364763,"[[59, 0, 21, 0], [10, 0, 70, 0], [8, 0, 232, 0...",0.743472,0.600000,0.318243,0.433333,0.364390,"[[16, 0, 4, 0], [2, 0, 18, 0], [4, 0, 56, 0], ..."
2,"{'n_estimators': 118, 'max_depth': 4, 'min_chi...",XGBoost,1.000000,1.00000,1.000000,1.000000,1.000000,"[[80, 0, 0, 0], [0, 80, 0, 0], [0, 0, 240, 0],...",0.806361,0.650000,0.604118,0.550000,0.552903,"[[17, 0, 3, 0], [3, 6, 10, 1], [2, 3, 51, 4], ..."
3,"{'n_estimators': 72, 'max_depth': 3, 'min_data...",Lighgbm,0.987427,0.87500,0.915638,0.827083,0.857884,"[[76, 0, 4, 0], [4, 58, 18, 0], [4, 1, 233, 2]...",0.782819,0.641667,0.593997,0.520833,0.507327,"[[17, 0, 2, 1], [3, 4, 12, 1], [3, 1, 53, 3], ..."
4,"{'n_estimators': 137, 'max_depth': 1, 'min_sam...",RandomForest,0.761339,0.50000,0.125000,0.250000,0.166667,"[[0, 0, 80, 0], [0, 0, 80, 0], [0, 0, 240, 0],...",0.705306,0.500000,0.125000,0.250000,0.166667,"[[0, 0, 20, 0], [0, 0, 20, 0], [0, 0, 60, 0], ..."


In [57]:
ronda_0.to_csv("vggnet16/gray_ronda_0.csv",sep=";")

## Resnet 

In [58]:
x_train_resnet, y_train_resnet, x_test_resnet, y_test_resnet = extract_features.features_resnet(path_gray)

******************************************************************** 
Extracción de caracteristicas por medio de RESNET
******************************************************************** 
Labels: ['cataratas' 'glaucoma' 'normal' 'retinopatia']
Labels encoded: [0 1 2 3]
******************************************************************** 
Extracción de caracteristicas por medio de RESNET finalizada
******************************************************************** 


In [59]:
models_cv = cross_validation.compare_models(x_train_resnet, y_train_resnet, 5)

******************************************************************** 
Iniciando torneo de modelos con validación cruzada
******************************************************************** 
Modelo: Random Forest
Modelo: Naive Bayes
Modelo: SVM
Modelo: LGBM
Modelo: XGBoost
******************************************************************** 
Finalizando torneo de modelos con validación cruzada
******************************************************************** 


In [60]:
models_cv

,Model,Train Accuracy,Train precision_macro,Train recall_macro,Train f1_macro,Train AUC-ROC,Test Accuracy,Test precision_macro,Test recall_macro,Test f1_macro,Test AUC-ROC
0,Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000,0.602083,0.538102,0.448958,0.436411,0.759023
1,Naive Bayes,0.685937,0.655432,0.644531,0.645250,0.853594,0.581250,0.556085,0.532292,0.531739,0.754449
2,SVM,0.631771,0.798796,0.455990,0.441751,0.919006,0.595833,0.425231,0.410417,0.371858,0.783364
3,LGBM,1.000000,1.000000,1.000000,1.000000,1.000000,0.606250,0.566785,0.492708,0.490284,0.756102
4,XGBoost,1.000000,1.000000,1.000000,1.000000,1.000000,0.591667,0.548200,0.475000,0.477808,0.760009


In [61]:
muestras_en_intervalo = 50
n_iter = 10

learning_ra_0 = list(set([float(x) for x in np.linspace(start = 0.001, stop = 0.8, num = muestras_en_intervalo)]))
estimadores_0 = list(set([int(x) for x in np.linspace(start = 10, stop = 200, num = muestras_en_intervalo)]))
profundidad_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 20, num = muestras_en_intervalo)]))
min_data_le_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 0.5*x_train_resnet.shape[0], num = muestras_en_intervalo)]))

parametros = {'learning_ra': learning_ra_0,
              'estimadores': estimadores_0,
              'profundidad': profundidad_0,
              'min_data_le': min_data_le_0
             }

In [62]:
ronda_0 = grid_search.train_models(x_train_resnet, y_train_resnet, x_test_resnet, y_test_resnet, n_iter, parametros)

******************************************************************** 
Iniciando busqueda de hiperparametros
******************************************************************** 
******************************************************************** 
Iteración: 0
******************************************************************** 
[LightGBM] [Warning] min_data_in_leaf is set=132, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=132
[LightGBM] [Warning] min_data_in_leaf is set=220, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=220
[LightGBM] [Warning] min_data_in_leaf is set=240, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=240
[LightGBM] [Warning] min_data_in_leaf is set=108, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=108
[LightGBM] [Warning] min_data_in_leaf is set=54, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=54
[LightGBM] [Warning] min_data_in_leaf is set=98, mi

In [63]:
ronda_0.head(5)

,parameters,modelo,auc_tr,accuracy_tr,precision_tr,recall_tr,f1_tr,Matriz_tr,auc_te,accuracy_te,precision_te,recall_te,f1_te,Matriz_te
0,"{'n_estimators': 91, 'max_depth': 4, 'min_data...",Lighgbm,1.000000,1.000000,1.000000,1.000000,1.000000,"[[80, 0, 0, 0], [0, 80, 0, 0], [0, 0, 240, 0],...",0.819847,0.616667,0.574668,0.516667,0.528257,"[[14, 1, 4, 1], [3, 6, 10, 1], [2, 7, 49, 2], ..."
1,"{'n_estimators': 165, 'max_depth': 8, 'min_sam...",RandomForest,0.923332,0.608333,0.590424,0.422917,0.376856,"[[55, 0, 25, 0], [8, 0, 72, 0], [5, 0, 235, 0]...",0.823889,0.600000,0.346659,0.416667,0.367103,"[[14, 0, 6, 0], [1, 0, 19, 0], [2, 0, 58, 0], ..."
2,"{'n_estimators': 118, 'max_depth': 4, 'min_chi...",XGBoost,1.000000,1.000000,1.000000,1.000000,1.000000,"[[80, 0, 0, 0], [0, 80, 0, 0], [0, 0, 240, 0],...",0.799417,0.591667,0.550787,0.495833,0.512237,"[[14, 1, 4, 1], [0, 5, 13, 2], [3, 5, 47, 5], ..."
3,"{'n_estimators': 72, 'max_depth': 3, 'min_data...",Lighgbm,0.977009,0.860417,0.877034,0.807292,0.834006,"[[73, 2, 2, 3], [3, 56, 20, 1], [3, 3, 232, 2]...",0.790792,0.566667,0.498738,0.475000,0.479987,"[[14, 1, 4, 1], [0, 3, 15, 2], [4, 5, 45, 6], ..."
4,"{'n_estimators': 137, 'max_depth': 1, 'min_sam...",RandomForest,0.805352,0.500000,0.125000,0.250000,0.166667,"[[0, 0, 80, 0], [0, 0, 80, 0], [0, 0, 240, 0],...",0.805278,0.500000,0.125000,0.250000,0.166667,"[[0, 0, 20, 0], [0, 0, 20, 0], [0, 0, 60, 0], ..."


In [64]:
ronda_0.to_csv("resnet/gray_ronda_0.csv",sep=";")

## NasNet 

In [65]:
x_train_nasnet, y_train_nasnet, x_test_nasnet, y_test_nasnet = extract_features.features_nasnet(path_gray)

******************************************************************** 
Feature extraction using NASNET
******************************************************************** 
Found 480 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
4/4 [==============================] - 29s 7s/step
Labels: ['cataratas' 'glaucoma' 'normal' 'retinopatia']
Labels encoded: [0 1 2 3]
******************************************************************** 
Feature extraction finished
******************************************************************** 


In [66]:
models_cv = cross_validation.compare_models(x_train_nasnet, y_train_nasnet, 5)

******************************************************************** 
Iniciando torneo de modelos con validación cruzada
******************************************************************** 
Modelo: Random Forest
Modelo: Naive Bayes
Modelo: SVM
Modelo: LGBM
Modelo: XGBoost
******************************************************************** 
Finalizando torneo de modelos con validación cruzada
******************************************************************** 


In [67]:
models_cv

,Model,Train Accuracy,Train precision_macro,Train recall_macro,Train f1_macro,Train AUC-ROC,Test Accuracy,Test precision_macro,Test recall_macro,Test f1_macro,Test AUC-ROC
0,Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000,0.472917,0.290217,0.248958,0.193367,0.508702
1,Naive Bayes,0.414583,0.440370,0.402083,0.362892,0.676152,0.300000,0.269162,0.279167,0.243168,0.523396
2,SVM,0.505208,0.269125,0.259375,0.186119,0.470701,0.497917,0.125000,0.248958,0.166432,0.521884
3,LGBM,1.000000,1.000000,1.000000,1.000000,1.000000,0.447917,0.254134,0.255208,0.223841,0.526727
4,XGBoost,1.000000,1.000000,1.000000,1.000000,1.000000,0.416667,0.252004,0.241667,0.219898,0.510365


In [68]:
muestras_en_intervalo = 50
n_iter = 10

learning_ra_0 = list(set([float(x) for x in np.linspace(start = 0.001, stop = 0.8, num = muestras_en_intervalo)]))
estimadores_0 = list(set([int(x) for x in np.linspace(start = 10, stop = 200, num = muestras_en_intervalo)]))
profundidad_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 20, num = muestras_en_intervalo)]))
min_data_le_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 0.5*x_train_nasnet.shape[0], num = muestras_en_intervalo)]))

parametros = {'learning_ra': learning_ra_0,
              'estimadores': estimadores_0,
              'profundidad': profundidad_0,
              'min_data_le': min_data_le_0
             }

In [69]:
ronda_0 = grid_search.train_models(x_train_nasnet, y_train_nasnet, x_test_nasnet, y_test_nasnet, n_iter, parametros)

******************************************************************** 
Iniciando busqueda de hiperparametros
******************************************************************** 
******************************************************************** 
Iteración: 0
******************************************************************** 
[LightGBM] [Warning] min_data_in_leaf is set=132, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=132
[LightGBM] [Warning] min_data_in_leaf is set=220, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=220
[LightGBM] [Warning] min_data_in_leaf is set=240, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=240
[LightGBM] [Warning] min_data_in_leaf is set=108, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=108
[LightGBM] [Warning] min_data_in_leaf is set=54, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=54
[LightGBM] [Warning] min_data_in_leaf is set=98, mi

In [70]:
ronda_0.head(5)

,parameters,modelo,auc_tr,accuracy_tr,precision_tr,recall_tr,f1_tr,Matriz_tr,auc_te,accuracy_te,precision_te,recall_te,f1_te,Matriz_te
0,"{'n_estimators': 91, 'max_depth': 4, 'min_data...",Lighgbm,1.000000,1.000000,1.000000,1.000000,1.000000,"[[80, 0, 0, 0], [0, 80, 0, 0], [0, 0, 240, 0],...",0.506847,0.425,0.263086,0.270833,0.256478,"[[4, 1, 13, 2], [4, 0, 16, 0], [6, 5, 44, 5], ..."
1,"{'n_estimators': 165, 'max_depth': 8, 'min_sam...",RandomForest,0.833220,0.500000,0.125000,0.250000,0.166667,"[[0, 0, 80, 0], [0, 0, 80, 0], [0, 0, 240, 0],...",0.502028,0.500,0.125000,0.250000,0.166667,"[[0, 0, 20, 0], [0, 0, 20, 0], [0, 0, 60, 0], ..."
2,"{'n_estimators': 118, 'max_depth': 4, 'min_chi...",XGBoost,1.000000,1.000000,1.000000,1.000000,1.000000,"[[80, 0, 0, 0], [0, 80, 0, 0], [0, 0, 240, 0],...",0.514222,0.400,0.229001,0.233333,0.216151,"[[1, 2, 15, 2], [2, 2, 13, 3], [5, 4, 44, 7], ..."
3,"{'n_estimators': 72, 'max_depth': 3, 'min_data...",Lighgbm,0.876103,0.589583,0.772751,0.392708,0.408338,"[[20, 0, 60, 0], [1, 6, 70, 3], [2, 0, 236, 2]...",0.484708,0.500,0.334420,0.266667,0.210181,"[[1, 0, 18, 1], [0, 0, 20, 0], [2, 0, 58, 0], ..."
4,"{'n_estimators': 137, 'max_depth': 1, 'min_sam...",RandomForest,0.647050,0.500000,0.125000,0.250000,0.166667,"[[0, 0, 80, 0], [0, 0, 80, 0], [0, 0, 240, 0],...",0.502611,0.500,0.125000,0.250000,0.166667,"[[0, 0, 20, 0], [0, 0, 20, 0], [0, 0, 60, 0], ..."


In [71]:
ronda_0.to_csv("nasnet/gray_ronda_0.csv",sep=";")

## Equalizer

## Vggnet16

In [72]:
x_train_vggnet, y_train_vggnet, x_test_vggnet, y_test_vggnet = extract_features.features_vgg16(path_equa)

******************************************************************** 
Extracción de caracteristicas por medio de VGGNET16
******************************************************************** 
Labels: ['cataratas' 'glaucoma' 'normal' 'retinopatia']
Labels encoded: [0 1 2 3]
4/4 [==============================] - 11s 3s/step
******************************************************************** 
Extracción de caracteristicas por medio de VGGNET16 finalizada
******************************************************************** 


In [ ]:
models_cv = cross_validation.compare_models(x_train_vggnet, y_train_vggnet, 5)

******************************************************************** 
Iniciando torneo de modelos con validación cruzada
******************************************************************** 
Modelo: Random Forest
Modelo: Naive Bayes
Modelo: SVM


In [ ]:
models_cv

In [ ]:
muestras_en_intervalo = 50
n_iter = 10

learning_ra_0 = list(set([float(x) for x in np.linspace(start = 0.001, stop = 0.8, num = muestras_en_intervalo)]))
estimadores_0 = list(set([int(x) for x in np.linspace(start = 10, stop = 200, num = muestras_en_intervalo)]))
profundidad_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 20, num = muestras_en_intervalo)]))
min_data_le_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 0.5*x_train_vggnet.shape[0], num = muestras_en_intervalo)]))

parametros = {'learning_ra': learning_ra_0,
              'estimadores': estimadores_0,
              'profundidad': profundidad_0,
              'min_data_le': min_data_le_0
             }

In [ ]:
ronda_0 = grid_search.train_models(x_train_vggnet, y_train_vggnet, x_test_vggnet, y_test_vggnet, n_iter, parametros)

In [ ]:
ronda_0.head(5)

In [ ]:
ronda_0.to_csv("vggnet16/equa_ronda_0.csv",sep=";")

## Resnet 

In [ ]:
x_train_resnet, y_train_resnet, x_test_resnet, y_test_resnet = extract_features.features_resnet(path_equa)

In [ ]:
models_cv = cross_validation.compare_models(x_train_resnet, y_train_resnet, 5)

In [ ]:
models_cv

In [ ]:
muestras_en_intervalo = 50
n_iter = 10

learning_ra_0 = list(set([float(x) for x in np.linspace(start = 0.001, stop = 0.8, num = muestras_en_intervalo)]))
estimadores_0 = list(set([int(x) for x in np.linspace(start = 10, stop = 200, num = muestras_en_intervalo)]))
profundidad_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 20, num = muestras_en_intervalo)]))
min_data_le_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 0.5*x_train_resnet.shape[0], num = muestras_en_intervalo)]))

parametros = {'learning_ra': learning_ra_0,
              'estimadores': estimadores_0,
              'profundidad': profundidad_0,
              'min_data_le': min_data_le_0
             }

In [ ]:
ronda_0 = grid_search.train_models(x_train_resnet, y_train_resnet, x_test_resnet, y_test_resnet, n_iter, parametros)

In [ ]:
ronda_0.head(5)

In [ ]:
ronda_0.to_csv("resnet/equa_ronda_0.csv",sep=";")

## NasNet 

In [ ]:
x_train_nasnet, y_train_nasnet, x_test_nasnet, y_test_nasnet = extract_features.features_nasnet(path_equa)

In [ ]:
models_cv = cross_validation.compare_models(x_train_nasnet, y_train_nasnet, 5)

In [ ]:
models_cv

In [ ]:
muestras_en_intervalo = 50
n_iter = 10

learning_ra_0 = list(set([float(x) for x in np.linspace(start = 0.001, stop = 0.8, num = muestras_en_intervalo)]))
estimadores_0 = list(set([int(x) for x in np.linspace(start = 10, stop = 200, num = muestras_en_intervalo)]))
profundidad_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 20, num = muestras_en_intervalo)]))
min_data_le_0 = list(set([int(x) for x in np.linspace(start = 1,  stop = 0.5*x_train_nasnet.shape[0], num = muestras_en_intervalo)]))

parametros = {'learning_ra': learning_ra_0,
              'estimadores': estimadores_0,
              'profundidad': profundidad_0,
              'min_data_le': min_data_le_0
             }

In [ ]:
ronda_0 = grid_search.train_models(x_train_nasnet, y_train_nasnet, x_test_nasnet, y_test_nasnet, n_iter, parametros)

In [ ]:
ronda_0.head(5)

In [ ]:
ronda_0.to_csv("nasnet/equa_ronda_0.csv",sep=";")